# Donación

Este es el último proyecto de aprendizaje supervisado. En este cuaderno, parte del código ya está implementado y tu tarea es implementar funcionalidad necesaria para completarlo. Adicionalmente, habrán preguntas que deberás responder relacionadas a tus implementaciones. Lee el contenido con cuidado.

En este proyecto, utilizarás diferentes algoritmos de aprendizaje supervisado para modelar los ingresos utilizando data de un censo de Estados Unidos de 1994. Después, tú decidirás los mejores candidatos de resultados preliminares y luego optimizarás del algoritmo. El objetivo en esta implementación es construir un modelo que prediga cuando un individuo gana más de $50,000. Entender los ingresos de individuos nos permitirá determinar qué número decir cuando pedimos donaciones. 

El dataset que utilizaremos viene del UCI Machine Learning Repository. Este dataset ha sido donado después de un artículo publicado.


## Exploración
Corre la siguiente celda para cargar las librerías necesarias de Python y cargar la información. Mira que la última columna, income, es nuestro label objetivo. El resto son features.

In [ ]:
# Importar librerías necesarias para el proyecto
import numpy as np
import pandas as pd
from time import time
from IPython.display import display

# Importamos visualizaciones ya implementadas
import visuals as vs

# Para mostrar gráficas en cuadernos
%matplotlib inline

# Cargamos info del csv
data = pd.read_csv("data/census.csv")

# Mostramos primer resultado
display(data.head(n=100))

Iniciaremos investigando el dataset para determinar cuántos individuos entran en cada grupo de salarios, y nos dirá el porcentaje de individuos haciendo más de \$50,000. En la siguiente celda necesitas programar lo siguiente:

* `n_records`: el número total de records.
* `n_greater_50k`: el número de individuos que ganan más de $50,000 al año.
* `n_at_most_50k`: el número de individuos que hacen como mucho \$50,000 al año.
* `greater_percent`: el porcentaje de individuos ganando más de \$50,000 al año.

In [ ]:
n_records = # TODO

n_greater_50k = # TODO

n_at_most_50k = # TODO

greater_percent = # TODO

# Print the results
print("Total number of records: {}".format(n_records))
print("Individuals making more than $50,000: {}".format(n_greater_50k))
print("Individuals making at most $50,000: {}".format(n_at_most_50k))
print("Percentage of individuals making more than $50,000: {}%".format(greater_percent))

### Exploración de features

* age: continuo.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* educatio: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuo.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: Black, White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other.
* sex: Female, Male.
* capital-gain: continuo.
* capital-loss: continuo.
* hours-per-week: continuo.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

## Preparando la información

Antes de poder utilizar la información como entrada de algoritmos de ML, es necesario limpiar la información, darle un formato, y restructurarla (preprocesamiento). En este caso, afortunadamente, no hay entradas que falten o inválidas con las que tengamos que lidiar. Aún así, hay ciertas cualidades que debemos ajustar que ayudarán mucho a los algoritmos.


### Transformando features continuos sesgados
En un dataset es común ver features cuyos valores siempre suelen estar cerca de un número individual, pero que también tienen resultados mucho más largos o mucho más pequeños. Los algoritmos son afectados a esos tipos de distribuciones y suelen desempeñarse peor si el rango no está normalizado. Los dos features con los que nos pasa esto son `capital-gain` y `capital-loss`. 

Haremos un histrograma de estos dos features:

In [ ]:
# Split the data into features and target label
income_raw = data['income']
features_raw = data.drop('income', axis = 1)

# Visualize skewed continuous features of original data
vs.distribution(data)

Para features con distribuciones tan sesgadas como `capital-gain` y `capital-loss`, es una práctica común aplicar transformación logarítmica para que los valores muy largos o muy pequeños no afecten negativamente el desempeño del algoritmo. Utilizar esta transformación reduce el rango de los valores causado por los outliers. Hay que tener cuidado porque el logaritmo de 0 no está definido, por lo que debemos transladar los valores a una pequeña cantidad sobre 0 para poderles aplicar la transformación.

In [ ]:
# Log-transform the skewed features
skewed = ['capital-gain', 'capital-loss']
features_log_transformed = pd.DataFrame(data = features_raw)
features_log_transformed[skewed] = features_raw[skewed].apply(lambda x: np.log(x + 1))

# Visualize the new log distributions
vs.distribution(features_log_transformed, transformed = True)

## Normalizando features numéricos

Además de transformar los features sesgados, es buena práctica escalar los features numéricos. Aplicar una escala no cambia la forma de la distribución de los features; normalizar asegura que cada feature sea tratado de la misma manera por un algoritmo de aprendizaje. Recuerda que una vez que apliquemos la transformación, las visualizaciones de la información ya no tendrán el mismo significado.

In [ ]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

features_log_minmax_transform = pd.DataFrame(data = features_log_transformed)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_transformed[numerical])

# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n = 5))

## Preprocesamiento de Data

De la tabla de arriba, podemos ver que muchos features son no numéricos (categóricos). Típicamente, los algoritmos de aprendizaje esperan entradas numéricas, por lo que convertiremos los otros features. Una manera popular de convertir variables categóricas es utilizando one-hot encoding, como hemos visto en redes neuronales también. 

Con one-hot encoding, creamos una variable "dummy" para cada posible categoría de los features categóricos. Por ejemplo, digamos que tenemos unFeature que tiene tres entradas posibles: A, B o C. Convertiremos esto a tres features: unFeatureA, unFeatureB y unFeatureC.

|            | Feature|
| ------------- |--------|
| 0      | B |  |
| 1      | C    |
| 2 | A         |

Se convierte en 

|            | A| B | C
| -------- |--------|--------|--------|
| 0      | 0 | 1 | 0 |
| 1      | 0 |0 |1    |
| 2      | 1 | 0| 0|

Adicionalmente, también tendremos que convertir el label por ser no numérico. En este caso, `income` puede ser ">50k" o "<=50k". Como sólo hay dos posibles valores, vamos a saltarnos el paso de hacer one-hot encoding y vamos a convertirlo a 0 y 1, respectivamente. 

En la siguiente celda implementaremos lo siguiente:
* Usa `pandas.get_dummies()` para aplicar one-hot encoding en `features_log_minmax_transform`.
* Convierte la etiqueta objetivo `income_raw` a entradas numéricas.

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

In [ ]:
# TODO: One-hot encode the 'features_log_minmax_transform' data using pandas.get_dummies()
features_final = # TODO

# TODO: Encode the 'income_raw' data to numerical values
income = # TODO

# Print the number of features after one-hot encoding
encoded = list(features_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))

print(encoded)
print(income[:20])

### Barajar y separar Data
Ya hemos convertidos todas las variables categóricas a features numéricos. Todos los features, adicionalmente, han sido normalizados. Como siempre, vamos a separar en training y testing (80% y 20%).

In [ ]:
# Import train_test_split
from sklearn.cross_validation import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    income, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

## Evaluando Desempeño del Modelo
En esta sección vamos a experimentar con cuatro algoritmos diferentes y determinaremos cuál modela mejor la información. Tres de estos son algoritmos supervisados que tú decidirás, y el cuarto será un predictor naive. 

### Métricas y Naive Predictor
Sabemos que es más probable que los individuos que ganan más de \$50,000 donen a caridad. Por lo tanto, estamos particularmente interesados en predecir quiénes hacen más de \$50,000 con exactitud (accuracy). Por lo tanto, parece más apropiado utilizar accuracy como métrica para evaluar el desempeño de cada modelo. Adicionalmente, identificar a alguien que hace menos de \$50,000 como alguien que lo hace puede afectar negativamente el problema que queremos solucionar. Por lo tanto, nos importa más la precisión que el recall. Por lo tanto, utilizaremos F-beta, la métrica que considera tanto precision como recall.

![f-beta](https://render.githubusercontent.com/render/math?math=F_%7B%5Cbeta%7D%20%3D%20%281%20%2B%20%5Cbeta%5E2%29%20%5Ccdot%20%5Cfrac%7Bprecision%20%5Ccdot%20recall%7D%7B%5Cleft%28%20%5Cbeta%5E2%20%5Ccdot%20precision%20%5Cright%29%20%2B%20recall%7D&mode=display "f-beta")

Cuando $\beta = 0.5$, se da más enfasis en la precisión. Esto se llama F$_{0.5}$ score.

Mirando la distribución de las clases, es claro que la mayoría de los individuos hacen menos de \$50,000. Esto puede afectar la exactitud: podríamos decir que nadie hace más de \$50,000, y, en general, el modelo estaría correcto. Sería un modelo exacto. Obviamente, esta es una decisión ingenua. Es importante iniciar estableciendo una predicción ingenua para poder establecer una base a partir de la cuál el modelo lo hará bien. 

**Recap**

**Accuracy**: mide qué tanto un clasificador hace predicciones correctas. Es la proporción entre predicciones correctas y el total de predicciones.

**Precision**: mide, de la proporción de mensajes clasificados como spam, cuántos en verdad lo eran. Es la porporción de positivos verdaderos a todos los positivos (todos los mensajes clasificados como spam):

```[True Positives/(True Positives + False Positives)]```

**Recall (sensitividad)**: mide, de la porporción de mensajes que eran spam, cuántos fueron clasificados como spam. Es la proporción de positivos verdaderos a todos los mensajes que realmente eran spam:

```[True Positives/(True Positives + False Negatives)]```

Para problemas de clasificación sesgados en sus distribuciones, como en este caso, la exactitud (accuracy) no es una buena métrica por lo que hemos discutido. Precisión y recall nos vienen bien. Combinamos ambas métricas en F1, que, como discutimos, el el promedio harmónica de ambos.

## Pregunta 1 - Desempeño de Predictor Ingenuo

Si elegimos un modelo que siempre predice que el individuo hizo más de $50,000, ¿cuál sería la exactitud y el F1-score en el dataset? Usa la siguiente celda.

**Nota**: Recuerda que el objetivo de generar un predictor ingenuo simplemente es ver lo que puede lograr un modelo iniciar sin inteligencia. En el mundo real, este predictor puede ser los resultados de un modelo previo, o los resultados de un paper de investigación que estás mejorando. Si no hay manera de establecer un modelo base, el primer objetivo suele hacer crear un modelo que lo haga mejor que un modelo aleatorio.

In [ ]:
# Recuerda que en income tenemos 0's y 1's. Si sumamos todos los valores, nos dan cuántos tienen >50K.
TP = np.sum(income) 

# Los falsos positivos son los que no tienen valor de 1. Es decir, el total - TP.
FP = income.count() - TP

# No se predicen negativos en este ejemplo
TN = 0 
FN = 0

accuracy = # TODO

# No hay falsos negativos
recall = # TODO

# [True Positives/(True Positives + False Positives)]
precision = # TODO

beta = 0.5

# TODO: Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
fscore = 

# Print the results 
print("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

## Modelos de Aprendizaje Supervisado

Estos son algunos de los modelos que hemos vistos y tenemos en scikit-learn:
* Gaussian Naive Bayes (GaussianNB)
* Decision Trees
* K-Nearest Neighbors (KKNeighbors)
* Stochastic Gradient Descent Classifier (SGDC)
* Support Vector Machines (SVM)
* Logistic Regression

## Pregunta 2 - Aplicando el modelo
Lista 3 modelos de aprendizaje supervisado que utilizarás para probar. Para cada modelo:
* Describe una aplicación del mundo real donde ese tipo de modelo se puede aplicr
* Cuáles son las fortalezas. ¿Dónde se desempeña bien?
* Cuáles son las debilidades. ¿Dónde no desempeña bien?
* ¿Por qué este modelo es un buen candidato para este problema?

** Respuesta** 


En la siguiente celda, vas a tener que:
* Importar `fbeta_score` y `accuracy_score` de `sklearn.metrics`.
* Entrenar el modelo con la data de entrenamiento y registrar el tiempo
* Hacer predicciones en el testing set y, adiconalmente, en los primeros 300 puntos de entrenamiento `X_train[:300]`.
* Registrar el tiempo total de predicción.
* Calcular el accuracy en training y testing.
* Calcular el F-score para ambos.

In [ ]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner = 
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = 
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test =
    predictions_train =
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] =
            
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = 
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['acc_test'] =
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()
    results['f_train'] = 
        
    # TODO: Compute F-score on the test set which is y_test
    results['f_test'] =  
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

En la siguiente celda, implementa lo siguiente:
* Importa los tres modelos de aprendizaje que discutiste en la anterior sección
* Inicializalos como clf_A, clf_B y clf_C.
* Asigna el parámetro random_state con el mismo valor para los modelos que lo permitan
* No asignes parámetros a tus modelos excepto el anterior. Luego ajustaremos un modelo específico.
* Calcula el número de registros igual a 1%, 10% y 100% del training set.

Se puede tomar un tiempo en entrenar el modelo.

In [ ]:
# TODO: Import the three supervised learning models from sklearn

# TODO: Initialize the three models
clf_A = # TODO
clf_B = # TODO
clf_C = # TODO

# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
# HINT: samples_100 is the entire training set i.e. len(y_train)
# HINT: samples_10 is 10% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
# HINT: samples_1 is 1% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
samples_100 = # TODO
samples_10 = # TODO
samples_1 = # TODO

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
vs.evaluate(results, accuracy, fscore)

## Mejorando resultados

En esta última sección, elegirás el mejor modelo. Después de esto, utilizarás la técnica de grid search con todo el training set para ajustar por lo menos un parámetro con el objetivo de mejorar el F-score

## Pregunta 3

Revisa las gráficas anteriores. Describe en un párrafo el comportamiento de cada modelo y sus ventajas. ¿Cuál es el modelo que consideras más apropiado? En tu discusión, menciona:
* Métricas - F score en el testing cuando se usa el 100% del training set
* Tiempo de entrenamiento
* Qué tan bueno es el algoritmo para tu información

**Respuesta**

## Pregunta 4

En uno o dos párafos explica de qué manera funciona el modelo que elegiste. Asegúrate de describir las mejores cualidades del modelo, por ejemplo, la manera en la que se entrena o de qué manera hace predicciones. Evita utilizar fórmulas o explicaciones complejas.

**Respuesta**

## Model Tuning

Es hora de ajustar los parámetros del modelo que elegiste. Utiliza grid search (`GridSearchCV`) con un parámetro mínimo con 3 valores diferentes. Utiliza todo el training set para este paso. En la siguiente celda, implementa:

* Importa `sklearn.grid_search.GridSearchCV` y `sklearn.metrics.make_scorer`
* Inicia el clasificador que elegiste en clf
* Elige el mismo random_state que antes
* crea un diccionario con los parámetros que quieres ajustar.
Ejemplo: `parameters = {'parameter' : [list of values]}`.
* Evita utilizar el parámetro max_features
* Utiliza `make_scorer` para crear un objeto que califique con `fbeta_score` con un $\beta = 0.5$.
* Aplica grid search en el clasificador `clf` utiizando el `scorer`, y guarda el resultado en `grid_obj`
* Haz un fit con X_train y y_train.

In [ ]:
# TODO: Import 'GridSearchCV', 'make_scorer', and any other necessary libraries

# TODO: Initialize the classifier
clf = 

# TODO: Create the parameters list you wish to tune, using a dictionary if needed.
# HINT: parameters = {'parameter_1': [value1, value2], 'parameter_2': [value1, value2]}
parameters = 

# TODO: Make an fbeta_score scoring object using make_scorer()
scorer = 

# TODO: Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = 

# TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = 

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))

## Pregunta 5 - Evaluación Final
* ¿Cuál fue el F-score y el accuracy de tu modelo con testing data? ¿Son mejores o peores que el modelo no optimizado?
* ¿Cómo se compara el modelo optimizado con el predictor ingenuo de la primera pregunta?

**Respuesta**

## Importancia de Features

Una tarea importante al manejar un dataset es determinar qué features tienen más poder predictivo. Al enfocarnos en la relación entre unos pocos features cruciales y el label objetivo, simplificamos lo que entendemos del fenómeno. Para este proyecto, significa que queremos identificar pocos features que importen mucho para identificar si un individuo hace más de \$50, 000.

Utilizaremos random forest, un clasificador que tiene un parámetro `feature_importance_attribute`, el cuál es una función que hace un ranking de la importancia de los features.

### Pregunta 6
Cuando exploramos la data, vimos que tenemos 13 features disponibles para cada entrada. De esos 13 features, ¿cuáles 
crees que sean los 5 más importantes para hacer una predicción? ¿En qué orden los rankearías?

**Respuesta**

En la siguiente celda implementaremos:
* Importaremos RandomForest 
* Lo entrenaremos con todo el training set
* Extraeremos la importancia de los features con `.feature_importances_`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# TODO: Train the supervised model on the training set 
model = 

# TODO: Extract the feature importances
importances =

# Plot
vs.feature_plot(importances, X_train, y_train)

## Pregunta 7

Observa la visualización. Esta despliega los 5 features más relevantes para esta tarea con un RandomForest.
* ¿De qué manera se comparan estos 5 features con los que propusiste en la sexta pregunta?
* ¿La visualización confirma tu hipótesis?
* ¿Por qué crees que estos features son más relevantes? (si es que son diferentes)

**Respuesta**

## Selección de features

¿Cómo se desempeña un modelo si sólo utilizamos los features importantes? Con menos features requeridos, esperaríamos que el modelo se entrenaría mucho más rápido. En la visualización se ve que los 5 features más importantes contribuyen más de la mitad a la importancia. Esto es una prueba que podemos reducir el espacio de features sin reducir nuestras métricas.

In [ ]:
# Import functionality for cloning a model
from sklearn.base import clone

# Reduce the feature space
X_train_reduced = X_train[X_train.columns.values[(np.argsort(importances)[::-1])[:5]]]
X_test_reduced = X_test[X_test.columns.values[(np.argsort(importances)[::-1])[:5]]]

# Train on the "best" model found from grid search earlier
clf = (clone(best_clf)).fit(X_train_reduced, y_train)

# Make new predictions
reduced_predictions = clf.predict(X_test_reduced)

# Report scores from the final model using both versions of data
print("Final Model trained on full data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))
print("\nFinal Model trained on reduced data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, reduced_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, reduced_predictions, beta = 0.5)))

## Pregunta 8
¿Cómo se compara el F-score y accuracy entre el modelo entrenado con menos features y el original? ¿Si el tiempo de entrenamiento fuera un factor, considerarías reducir la la cantidad de información?

**Respuesta**